In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle 

from keras.models import Model 
from keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from keras.layers import Dropout, BatchNormalization, Activation
from keras.regularizers import l2
from keras.optimizers import SGD, Adam

2023-04-07 10:42:34.823375: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_csv("./data/edited_ratings.csv")

In [4]:
N = df.userId.max() + 1 # number of users 
M = df.movie_idx.max() + 1 # number of movies 

# split in train and test 
df = shuffle(df) 
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff] 
df_test = df.iloc[cutoff:] 

# init vars 
K = 10 # latent dimensionality 
mu = df_train.rating.mean() 
epoch = 25
reg = 0 # regularization penalty

# keras model 
u = Input(shape=(1, ))
m = Input(shape=(1, ))
u_embedding = Embedding(N, K)(u) # (N, 1, K) 
m_embedding = Embedding(M, K)(m) # (N, 1, K) 
u_embedding = Flatten()(u_embedding) # (N, K) 
m_embedding = Flatten()(m_embedding) # (N, K) 
x = Concatenate()([u_embedding, m_embedding]) # (N, 2K) 


2023-04-07 10:42:48.269574: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# the neural network 
x = Dense(400)(x) 
x = Activation('relu')(x)
x = Dense(1)(x)

In [6]:
model = Model(inputs=[u, m], outputs = x) 
model.compile(
    loss= 'mse', 
    optimizer=SGD(lr=0.01, momentum=0.9), 
    metrics=['mse']
)

/Users/faa/anaconda3/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [7]:
r = model.fit(x = [df_train.userId.values, df_train.movie_idx.values],
               y= df_train.rating.values - mu,
               epochs=epoch, 
               batch_size=128, 
               validation_data=(
                [df_test.userId.values, df_test.movie_idx.values], 
                df_test.rating.values - mu 
               ))

Epoch 1/25
  1215/125002 [..............................] - ETA: 2:13 - loss: 1.0618 - mse: 1.0618

InvalidArgumentError: Graph execution error:

Detected at node 'model/embedding/embedding_lookup' defined at (most recent call last):
    File "/Users/faa/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/faa/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/faa/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/faa/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/faa/anaconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/Users/faa/anaconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/Users/faa/anaconda3/lib/python3.10/asyncio/base_eve

In [ ]:
# plot losses 
plt.plot(r.history['loss'], label="train loss")
plt.plot(r.history['val_loss'], label='test loss')
plt.legend()
plt.show()

In [ ]:
# plot mse 
plt.plot(r.history['mean_squared_error'], label='train mse')
plt.plot(r.history['val_mean_squared_error'], label="test mse")
plt.legend()
plt.show()